# Fully operational 2D-PTV of the inertial particle settling through a TNTI

In [1]:
import openptv_python as optv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path
import os

In [2]:
# working directory
exp_dir = Path('../')
# os.chdir(exp_dir)
# os.listdir('.')
os.listdir(exp_dir)

['test_openptv_python_crossing_particle',
 'parameters',
 'cal',
 'img',
 'res',
 'C001H001S0005 (copy)']

In [3]:
# read parameters
from openptv_python.parameters import read_control_par

parameters_dir = 'parameters'
os.listdir(exp_dir / parameters_dir)


['multi_planes.par',
 'criteria.par',
 'pft_version',
 'track.par',
 'detect_plate.par',
 'sequence.par',
 'shaking.par',
 'man_ori.par',
 'orient.par',
 'targ_rec.par',
 'examine.par',
 'cal_ori.par',
 'unsharp_mask.par',
 'ptv.par',
 'pft_version.par',
 'dumbbell.par',
 'man_ori.dat',
 'sortgrid.par']

In [6]:

from openptv_python.parameters import read_control_par
cpar = read_control_par(os.path.join(exp_dir, parameters_dir,'ptv.par'))
print(cpar)

ControlPar(num_cams=1, img_base_name=['img/C001H001S0005000001.tif'], cal_img_base_name=['cal/cam1.tif'], hp_flag=1, all_cam_flag=0, tiff_flag=1, imx=1024, imy=1024, pix_x=0.001, pix_y=0.001, chfield=0, mm=MultimediaPar(nlay=1, n1=1.0, n2=[1.0], d=[1.0], n3=1.0))


In [ ]:
from openptv_python.calibration import Calibration, read_calibration
ori_file = exp_dir / 'cal' / 'cam1.tif.ori'
addpar_file = exp_dir / 'cal' / 'cam1.tif.addpar'
cal = [read_calibration(ori_file, addpar_file)]
print(cal)

ValueError: mutable default <class 'numpy.ndarray'> for field origin is not allowed: use default_factory

In [ ]:
from openptv_python.track import (
    track_forward_start,
    trackback_c,
    trackcorr_c_finish,
    trackcorr_c_loop,
)
from openptv_python.tracking_run import tr_new

In [ ]:
from openptv_python.parameters import read_sequence_par, read_track_par, read_volume_par

seq_par_fname = "parameters/sequence.par"
tpar_fname = "parameters/track.par"
vpar_fname = "parameters/criteria.par"
cpar_fname = "parameters/ptv.par"

seq_par = read_sequence_par(seq_par_fname, cpar.num_cams)
tpar = read_track_par(tpar_fname)
vpar = read_volume_par(vpar_fname)

print(seq_par)
print(tpar)
print(vpar)

In [ ]:
# image segmentation
img_name = seq_par.img_base_name[0] % 1
print(img_name)
img = mpimg.imread(img_name)
plt.imshow(img, cmap='gray')

In [ ]:
# prepare image for detection
from openptv_python.image_processing import (
    prepare_image
)   
img_hp = prepare_image(img, 1, 0)
plt.imshow(img_hp, cmap='gray')

In [ ]:
# detection
from openptv_python.parameters import TargetPar
from openptv_python.segmentation import target_recognition

tpar = TargetPar(
    gvthresh=[20, 255, 255, 255],
    discont=5,
    nnmin=1,
    nnmax=10,
    sumg_min=12,
    nxmin=1,
    nxmax=10,
    nymin=1,
    nymax=10,
)

target_array = target_recognition(img, tpar, 0, cpar)

print(len(target_array))
print(target_array[0].count_pixels())



In [ ]:
plt.imshow(img_hp, cmap='gray')
for target in target_array:
    plt.plot(target.x, target.y, 'r+')

In [ ]:

run = tr_new(
            seq_par_fname,
            tpar_fname,
            vpar_fname,
            cpar_fname,
            4,
            20000,
            "res/rt_is",
            "res/ptv_is",
            "res/added",
            cal,
            10000.0,
)

run.tpar.add = 0
print(f"run.seq_par.first = {run.seq_par.first} run.seq_par.last = {run.seq_par.last}")

track_forward_start(run)


In [ ]:
trackcorr_c_loop(run, run.seq_par.first)

for step in range(run.seq_par.first + 1, run.seq_par.last):
    # print(f"step = {step}")
    trackcorr_c_loop(run, step)
    # print(f"run.npart = {run.npart}")
    # print(f"run.nlinks = {run.nlinks}")

trackcorr_c_finish(run, run.seq_par.last)


range_val = run.seq_par.last - run.seq_par.first
npart = run.npart / range_val
nlinks = run.nlinks / range_val
